# Power Flow Calculations Without Main Field Losses

The test bench consists of three nodes, a three winding transformer and two loads at the medium and low voltage port of
the transformer. We sweep through tap positions and active power consumption of the loads, respectively.
This is meant to get an insight into the results of the transformer, that are achieved with pandapower.

As the power rating of the high voltage port is 300 MVA, that of the medium voltage port 300 MVA as well and 100 MVA for
the low voltage port, the following applies:
The medium voltage load is varied within the range of +/- 300 MVA.
The low voltage load is only varied within the permissible range, that is left.
E.g. if the medium voltage load is 250 MVA, ist only is varied from -100 MVA to 50 MVA.

In [1]:
# Specify information about persisted entities
import json
import os

from calculation.pandapower.ThreeWindingTestBench import ThreeWindingTestBench
from encoder import CustomDecoder
from encoder.DictEncoder import DictEncoder

result_directory = os.path.join("..", "..", "results", "three_winding")
result_file = os.path.join(result_directory, "withoutMainFieldLosses.json")

# Define basic parameters
s_rated_hv_mva: float = 300
s_rated_mv_mva: float = 300
s_rated_lv_mva: float = 100
v_ref_kv: float = 380
s_ref_mva: float = 300
tap_min: int = -10
tap_max: int = 10
p_step_size: float = 0.1  # in p.u.

if os.path.exists(result_file):
    # The result file exists. Read it and use it
    print("Pre-calculated results are available. Load them.")
    with open(result_file, "r") as file_to_read:
        json_string = file_to_read.read()
        results = json.loads(json_string, object_hook=CustomDecoder.custom_decode)
else:
    # Result file doesn't exist. Perform simulations and write the file
    results = ThreeWindingTestBench().calculate(tap_min=tap_min, tap_max=tap_max, s_nom_hv_mva=s_rated_hv_mva,
                                                s_nom_mv_mva=s_rated_mv_mva, s_nom_lv_mva=s_rated_lv_mva,
                                                p_step_pu=p_step_size, v_ref_kv=v_ref_kv, s_ref_mva=s_ref_mva,
                                                tap_at_star_point=False)

    if not os.path.exists(result_directory):
        os.makedirs(result_directory)
    with open(result_file, "w") as file_to_write_to:
        json.dump(results, file_to_write_to, cls=DictEncoder, indent=2)

2021-04-19 15:30:12,430,430 root INFO - Starting to calculate grid with pandapower. Parameters: tap = -10...10, reference = 300.00 MVA @ 380.00 kV, tap changer is not at star point
